In [1]:
from sympy import *
init_printing(use_latex='mathjax')

## Set up tensors so that we may use reasonable math notation

In [2]:
from sympy import MutableDenseNDimArray as MArray

In [3]:
class MyMArray(MArray):
    
    def __mul__(self, other):
        try:
            return super().__mul__(other)
        except ValueError:
            if not isinstance(other, MyMArray):
                raise ValueError("Need MyMArray for contraction")
            
            rank_self = self.rank()
            prod = tensorproduct(self, other)
            return MyMArray( tensorcontraction(prod, (rank_self - 1, rank_self)) )
    
    def __matmul__(self, other):
        try:
            return super().__mul__(other)
        except ValueError:
            if not isinstance(other, MyMArray):
                raise ValueError("Need MyMArray for tensor product")

            return MyMArray( tensorproduct(self, other) )
    
    def __pow__(self, other):
        try:
            return super().__mul__(other)
        except ValueError:
            if not isinstance(other, MyMArray):
                raise ValueError("Need MyMArray for double contraction")

            rank_self = self.rank()
            prod = tensorproduct(self, other)
            contract = tensorcontraction(prod, (rank_self - 1, rank_self))
            return MyMArray( tensorcontraction(contract, (rank_self - 2, rank_self - 1)) )
        
    def ip(self, other):
        if not isinstance(other, MyMArray):
            raise ValueError("Need MyMArray for inner product")
        
        rank_self = self.rank()
        prod = tensorproduct(self, other)
        while rank_self > 0:
            prod = tensorcontraction(prod, (0, rank_self))
            rank_self -= 1
            
        return prod

## Set up necessary objects that weak forms will be written in

In [37]:
vec_dim = 5
mat_dim = 3

x, y, z = symbols('x y z')
xi = MyMArray([x, y, z])

alpha, dt, L2, L3 = symbols(r'\alpha \delta\ t L_2 L_3')

Q_vec = zeros(vec_dim, 1)
for i in range(vec_dim):
    Q_vec[i] = Function('Q_' + str(i + 1))(x, y, z)
    
Q0_vec = zeros(vec_dim, 1)
for i in range(vec_dim):
    Q0_vec[i] = Function('Q_{0' + str(i + 1) + '}')(x, y, z)

phi_vec = zeros(vec_dim, 1)
for i in range(vec_dim):
    phi_vec[i] = Function(r'\phi_' + str(i + 1))(x, y, z)
    
phi_i = Function(r'\phi_i')(x, y, z)
phi_j = Function(r'\phi_j')(x, y, z)
    
Lambda_vec = zeros(vec_dim, 1)
for i in range(vec_dim):
    Lambda_vec[i] = Function(r'\Lambda_' + str(i + 1))(x, y, z)
    
display([alpha, dt, L2, L3])
display(xi)
display(Q_vec)
display(Q0_vec)
display(phi_vec)
display(Lambda_vec)

display(phi_i)
display(phi_j)

[\alpha, \delta t, L₂, L₃]

[x  y  z]

⎡Q₁(x, y, z)⎤
⎢           ⎥
⎢Q₂(x, y, z)⎥
⎢           ⎥
⎢Q₃(x, y, z)⎥
⎢           ⎥
⎢Q₄(x, y, z)⎥
⎢           ⎥
⎣Q₅(x, y, z)⎦

⎡Q_{01}(x, y, z)⎤
⎢               ⎥
⎢Q_{02}(x, y, z)⎥
⎢               ⎥
⎢Q_{03}(x, y, z)⎥
⎢               ⎥
⎢Q_{04}(x, y, z)⎥
⎢               ⎥
⎣Q_{05}(x, y, z)⎦

⎡\phi₁(x, y, z)⎤
⎢              ⎥
⎢\phi₂(x, y, z)⎥
⎢              ⎥
⎢\phi₃(x, y, z)⎥
⎢              ⎥
⎢\phi₄(x, y, z)⎥
⎢              ⎥
⎣\phi₅(x, y, z)⎦

⎡\Lambda₁(x, y, z)⎤
⎢                 ⎥
⎢\Lambda₂(x, y, z)⎥
⎢                 ⎥
⎢\Lambda₃(x, y, z)⎥
⎢                 ⎥
⎢\Lambda₄(x, y, z)⎥
⎢                 ⎥
⎣\Lambda₅(x, y, z)⎦

\phiᵢ(x, y, z)

\phi_j(x, y, z)

## Choose basis

In [5]:
basis = []
basis.append(MyMArray([[1, 0, 0],
                   [0, 0, 0],
                   [0, 0, -1]]))
basis.append(MyMArray([[0, 1, 0],
                   [1, 0, 0],
                   [0, 0, 0]]))
basis.append(MyMArray([[0, 0, 1],
                   [0, 0, 0],
                   [1, 0, 0]]))
basis.append(MyMArray([[0, 0, 0],
                   [0, 1, 0],
                   [0, 0, -1]]))
basis.append(MyMArray([[0, 0, 0],
                   [0, 0, 1],
                   [0, 1, 0]]))

display(basis)

⎡⎡1  0  0 ⎤  ⎡0  1  0⎤  ⎡0  0  1⎤  ⎡0  0  0 ⎤  ⎡0  0  0⎤⎤
⎢⎢        ⎥  ⎢       ⎥  ⎢       ⎥  ⎢        ⎥  ⎢       ⎥⎥
⎢⎢0  0  0 ⎥, ⎢1  0  0⎥, ⎢0  0  0⎥, ⎢0  1  0 ⎥, ⎢0  0  1⎥⎥
⎢⎢        ⎥  ⎢       ⎥  ⎢       ⎥  ⎢        ⎥  ⎢       ⎥⎥
⎣⎣0  0  -1⎦  ⎣0  0  0⎦  ⎣1  0  0⎦  ⎣0  0  -1⎦  ⎣0  1  0⎦⎦

In [6]:
# basis = []
# basis.append(MyMArray([[2/sqrt(3), 0, 0],
#                        [0, -1/sqrt(3), 0],
#                        [0, 0, -1/sqrt(3)]]))
# basis.append(MyMArray([[0, 0, 0],
#                    [0, 1, 0],
#                    [0, 0, -1]]))
# basis.append(MyMArray([[0, 1, 0],
#                    [1, 0, 0],
#                    [0, 0, 0]]))
# basis.append(MyMArray([[0, 0, 1],
#                    [0, 0, 0],
#                    [1, 0, 0]]))
# basis.append(MyMArray([[0, 0, 0],
#                    [0, 0, 1],
#                    [0, 1, 0]]))

# display(basis)

## Construct matrix objects from basis

In [38]:
Q = MyMArray.zeros(mat_dim, mat_dim)
for i in range(vec_dim):
    Q += basis[i]*Q_vec[i]
    
Q0 = MyMArray.zeros(mat_dim, mat_dim)
for i in range(vec_dim):
    Q0 += basis[i]*Q0_vec[i]
    
Lambda = MyMArray.zeros(mat_dim, mat_dim)
for i in range(vec_dim):
    Lambda += basis[i]*Lambda_vec[i]
    
Phi_i = []
for i in range(vec_dim):
    Phi_i.append( MyMArray(basis[i]*phi_i) )
    
Phi_j = []
for i in range(vec_dim):
    Phi_j.append( MyMArray(basis[i]*phi_j) )

display(Q)
display(Lambda)
display(Phi_i)
display(Phi_j)

⎡Q₁(x, y, z)  Q₂(x, y, z)         Q₃(x, y, z)        ⎤
⎢                                                    ⎥
⎢Q₂(x, y, z)  Q₄(x, y, z)         Q₅(x, y, z)        ⎥
⎢                                                    ⎥
⎣Q₃(x, y, z)  Q₅(x, y, z)  -Q₁(x, y, z) - Q₄(x, y, z)⎦

⎡\Lambda₁(x, y, z)  \Lambda₂(x, y, z)            \Lambda₃(x, y, z)           ⎤
⎢                                                                            ⎥
⎢\Lambda₂(x, y, z)  \Lambda₄(x, y, z)            \Lambda₅(x, y, z)           ⎥
⎢                                                                            ⎥
⎣\Lambda₃(x, y, z)  \Lambda₅(x, y, z)  -\Lambda₁(x, y, z) - \Lambda₄(x, y, z)⎦

⎡⎡\phiᵢ(x, y, z)  0         0       ⎤  ⎡      0         \phiᵢ(x, y, z)  0⎤  ⎡ 
⎢⎢                                  ⎥  ⎢                                 ⎥  ⎢ 
⎢⎢      0         0         0       ⎥, ⎢\phiᵢ(x, y, z)        0         0⎥, ⎢ 
⎢⎢                                  ⎥  ⎢                                 ⎥  ⎢ 
⎣⎣      0         0  -\phiᵢ(x, y, z)⎦  ⎣      0               0         0⎦  ⎣\

     0         0  \phiᵢ(x, y, z)⎤  ⎡0        0                0       ⎤  ⎡0   
                                ⎥  ⎢                                  ⎥  ⎢    
     0         0        0       ⎥, ⎢0  \phiᵢ(x, y, z)         0       ⎥, ⎢0   
                                ⎥  ⎢                                  ⎥  ⎢    
phiᵢ(x, y, z)  0        0       ⎦  ⎣0        0         -\phiᵢ(x, y, z)⎦  ⎣0  \

     0               0       ⎤⎤
                             ⎥⎥
     0         \phiᵢ(x, y, z)⎥⎥
                             ⎥⎥
phiᵢ(x, y, z)        0       ⎦⎦

⎡⎡\phi_j(x, y, z)  0         0        ⎤  ⎡       0         \phi_j(x, y, z)  0⎤
⎢⎢                                    ⎥  ⎢                                   ⎥
⎢⎢       0         0         0        ⎥, ⎢\phi_j(x, y, z)         0         0⎥
⎢⎢                                    ⎥  ⎢                                   ⎥
⎣⎣       0         0  -\phi_j(x, y, z)⎦  ⎣       0                0         0⎦

  ⎡       0         0  \phi_j(x, y, z)⎤  ⎡0         0                0        
  ⎢                                   ⎥  ⎢                                    
, ⎢       0         0         0       ⎥, ⎢0  \phi_j(x, y, z)         0        
  ⎢                                   ⎥  ⎢                                    
  ⎣\phi_j(x, y, z)  0         0       ⎦  ⎣0         0         -\phi_j(x, y, z)

⎤  ⎡0         0                0       ⎤⎤
⎥  ⎢                                   ⎥⎥
⎥, ⎢0         0         \phi_j(x, y, z)⎥⎥
⎥  ⎢                                   ⎥⎥
⎦  ⎣0  \phi_j(x, y, z)         0       ⎦

In [8]:
grad_Phi_i = []
for i in range(vec_dim):
    grad_Phi_i.append( MyMArray(derive_by_array(Phi_i[i], xi)) )
    
grad_Phi_j = []
for j in range(vec_dim):
    grad_Phi_j.append( MyMArray(derive_by_array(Phi_j[j], xi)) )
    
display(grad_Phi_i)
display(grad_Phi_j)

⎡⎡⎡∂                                         ⎤  ⎡∂                            
⎢⎢⎢──(\phiᵢ(x, y, z))  0           0         ⎥  ⎢──(\phiᵢ(x, y, z))  0        
⎢⎢⎢∂x                                        ⎥  ⎢∂y                           
⎢⎢⎢                                          ⎥  ⎢                             
⎢⎢⎢        0           0           0         ⎥  ⎢        0           0        
⎢⎢⎢                                          ⎥  ⎢                             
⎢⎢⎢                        ∂                 ⎥  ⎢                        ∂    
⎢⎢⎢        0           0  -──(\phiᵢ(x, y, z))⎥  ⎢        0           0  -──(\p
⎣⎣⎣                        ∂x                ⎦  ⎣                        ∂y   

             ⎤  ⎡∂                                         ⎤⎤  ⎡⎡             
   0         ⎥  ⎢──(\phiᵢ(x, y, z))  0           0         ⎥⎥  ⎢⎢        0    
             ⎥  ⎢∂z                                        ⎥⎥  ⎢⎢             
             ⎥  ⎢                                  

⎡⎡⎡∂                                           ⎤  ⎡∂                          
⎢⎢⎢──(\phi_j(x, y, z))  0           0          ⎥  ⎢──(\phi_j(x, y, z))  0     
⎢⎢⎢∂x                                          ⎥  ⎢∂y                         
⎢⎢⎢                                            ⎥  ⎢                           
⎢⎢⎢         0           0           0          ⎥  ⎢         0           0     
⎢⎢⎢                                            ⎥  ⎢                           
⎢⎢⎢                         ∂                  ⎥  ⎢                         ∂ 
⎢⎢⎢         0           0  -──(\phi_j(x, y, z))⎥  ⎢         0           0  -──
⎣⎣⎣                         ∂x                 ⎦  ⎣                         ∂y

                 ⎤  ⎡∂                                           ⎤⎤  ⎡⎡       
      0          ⎥  ⎢──(\phi_j(x, y, z))  0           0          ⎥⎥  ⎢⎢       
                 ⎥  ⎢∂z                                          ⎥⎥  ⎢⎢       
                 ⎥  ⎢                              

In [9]:
div_Phi_i = []
for i in range(vec_dim):
    div_Phi_i.append( MyMArray(tensorcontraction(grad_Phi_i[i], (0, 1))) )
    
div_Phi_j = []
for i in range(vec_dim):
    div_Phi_j.append( MyMArray(tensorcontraction(grad_Phi_j[i], (0, 1))) )
    
display(div_Phi_i)
display(div_Phi_j)

⎡⎡∂                       ∂                 ⎤  ⎡∂                   ∂         
⎢⎢──(\phiᵢ(x, y, z))  0  -──(\phiᵢ(x, y, z))⎥, ⎢──(\phiᵢ(x, y, z))  ──(\phiᵢ(x
⎣⎣∂x                      ∂z                ⎦  ⎣∂y                  ∂x        

           ⎤  ⎡∂                      ∂                 ⎤  ⎡   ∂              
, y, z))  0⎥, ⎢──(\phiᵢ(x, y, z))  0  ──(\phiᵢ(x, y, z))⎥, ⎢0  ──(\phiᵢ(x, y, 
           ⎦  ⎣∂z                     ∂x                ⎦  ⎣   ∂y             

      ∂                 ⎤  ⎡   ∂                   ∂                 ⎤⎤
z))  -──(\phiᵢ(x, y, z))⎥, ⎢0  ──(\phiᵢ(x, y, z))  ──(\phiᵢ(x, y, z))⎥⎥
      ∂z                ⎦  ⎣   ∂z                  ∂y                ⎦⎦

⎡⎡∂                        ∂                  ⎤  ⎡∂                    ∂      
⎢⎢──(\phi_j(x, y, z))  0  -──(\phi_j(x, y, z))⎥, ⎢──(\phi_j(x, y, z))  ──(\phi
⎣⎣∂x                       ∂z                 ⎦  ⎣∂y                   ∂x     

               ⎤  ⎡∂                       ∂                  ⎤  ⎡   ∂        
_j(x, y, z))  0⎥, ⎢──(\phi_j(x, y, z))  0  ──(\phi_j(x, y, z))⎥, ⎢0  ──(\phi_j
               ⎦  ⎣∂z                      ∂x                 ⎦  ⎣   ∂y       

             ∂                  ⎤  ⎡   ∂                    ∂                 
(x, y, z))  -──(\phi_j(x, y, z))⎥, ⎢0  ──(\phi_j(x, y, z))  ──(\phi_j(x, y, z)
             ∂z                 ⎦  ⎣   ∂z                   ∂y                

 ⎤⎤
)⎥⎥
 ⎦⎦

In [10]:
grad_Q = MyMArray(derive_by_array(Q, xi))
display(grad_Q)

⎡⎡∂                ∂                          ∂                        ⎤  ⎡∂  
⎢⎢──(Q₁(x, y, z))  ──(Q₂(x, y, z))            ──(Q₃(x, y, z))          ⎥  ⎢──(
⎢⎢∂x               ∂x                         ∂x                       ⎥  ⎢∂y 
⎢⎢                                                                     ⎥  ⎢   
⎢⎢∂                ∂                          ∂                        ⎥  ⎢∂  
⎢⎢──(Q₂(x, y, z))  ──(Q₄(x, y, z))            ──(Q₅(x, y, z))          ⎥  ⎢──(
⎢⎢∂x               ∂x                         ∂x                       ⎥  ⎢∂y 
⎢⎢                                                                     ⎥  ⎢   
⎢⎢∂                ∂                  ∂                 ∂              ⎥  ⎢∂  
⎢⎢──(Q₃(x, y, z))  ──(Q₅(x, y, z))  - ──(Q₁(x, y, z)) - ──(Q₄(x, y, z))⎥  ⎢──(
⎣⎣∂x               ∂x                 ∂x                ∂x             ⎦  ⎣∂y 

              ∂                          ∂                        ⎤  ⎡∂       
Q₁(x, y, z))  ──(Q₂(x, y, z))            ──(Q₃(x, y

In [11]:
div_Q = MyMArray( tensorcontraction(grad_Q, (0, 1)) )
display(div_Q)

⎡∂                 ∂                 ∂                ∂                 ∂     
⎢──(Q₁(x, y, z)) + ──(Q₂(x, y, z)) + ──(Q₃(x, y, z))  ──(Q₂(x, y, z)) + ──(Q₄(
⎣∂x                ∂y                ∂z               ∂x                ∂y    

            ∂                  ∂                 ∂                 ∂          
x, y, z)) + ──(Q₅(x, y, z))  - ──(Q₁(x, y, z)) + ──(Q₃(x, y, z)) - ──(Q₄(x, y,
            ∂z                 ∂z                ∂x                ∂z         

       ∂              ⎤
 z)) + ──(Q₅(x, y, z))⎥
       ∂y             ⎦

## Calculate each of the terms in the residual vector

In [12]:
R1 = zeros(vec_dim, 1)
for i in range(vec_dim):
    R1[i, 0] = Phi_i[i].ip(Q)

R1 = simplify(R1)
display(R1)

⎡(2⋅Q₁(x, y, z) + Q₄(x, y, z))⋅\phiᵢ(x, y, z)⎤
⎢                                            ⎥
⎢        2⋅Q₂(x, y, z)⋅\phiᵢ(x, y, z)        ⎥
⎢                                            ⎥
⎢        2⋅Q₃(x, y, z)⋅\phiᵢ(x, y, z)        ⎥
⎢                                            ⎥
⎢(Q₁(x, y, z) + 2⋅Q₄(x, y, z))⋅\phiᵢ(x, y, z)⎥
⎢                                            ⎥
⎣        2⋅Q₅(x, y, z)⋅\phiᵢ(x, y, z)        ⎦

In [40]:
R2 = zeros(vec_dim, 1)
for i in range(vec_dim):
    R2[i, 0] = -(1 + alpha * dt) * Phi_i[i].ip(Q0)

R2 = simplify(R2)
display(R2)

⎡(\alpha⋅\delta t + 1)⋅(-2⋅Q_{01}(x, y, z) - Q_{04}(x, y, z))⋅\phiᵢ(x, y, z)⎤
⎢                                                                           ⎥
⎢          -2⋅(\alpha⋅\delta t + 1)⋅Q_{02}(x, y, z)⋅\phiᵢ(x, y, z)          ⎥
⎢                                                                           ⎥
⎢          -2⋅(\alpha⋅\delta t + 1)⋅Q_{03}(x, y, z)⋅\phiᵢ(x, y, z)          ⎥
⎢                                                                           ⎥
⎢(\alpha⋅\delta t + 1)⋅(-Q_{01}(x, y, z) - 2⋅Q_{04}(x, y, z))⋅\phiᵢ(x, y, z)⎥
⎢                                                                           ⎥
⎣          -2⋅(\alpha⋅\delta t + 1)⋅Q_{05}(x, y, z)⋅\phiᵢ(x, y, z)          ⎦

In [59]:
R3 = zeros(vec_dim, 1)
for i in range(vec_dim):
    R3[i, 0] = -dt * (-Phi_i[i].ip(Lambda))

R3 = simplify(R3)
display(R3)

⎡\delta t⋅(2⋅\Lambda₁(x, y, z) + \Lambda₄(x, y, z))⋅\phiᵢ(x, y, z)⎤
⎢                                                                 ⎥
⎢           2⋅\delta t⋅\Lambda₂(x, y, z)⋅\phiᵢ(x, y, z)           ⎥
⎢                                                                 ⎥
⎢           2⋅\delta t⋅\Lambda₃(x, y, z)⋅\phiᵢ(x, y, z)           ⎥
⎢                                                                 ⎥
⎢\delta t⋅(\Lambda₁(x, y, z) + 2⋅\Lambda₄(x, y, z))⋅\phiᵢ(x, y, z)⎥
⎢                                                                 ⎥
⎣           2⋅\delta t⋅\Lambda₅(x, y, z)⋅\phiᵢ(x, y, z)           ⎦

In [13]:
E1 = zeros(vec_dim, 1)
for i in range(vec_dim):
    E1[i, 0] = -grad_Phi_i[i].ip(grad_Q)
    
E1 = simplify(E1)
display(E1)

⎡    ∂               ∂                      ∂               ∂                 
⎢- 2⋅──(Q₁(x, y, z))⋅──(\phiᵢ(x, y, z)) - 2⋅──(Q₁(x, y, z))⋅──(\phiᵢ(x, y, z))
⎢    ∂x              ∂x                     ∂y              ∂y                
⎢                                                                             
⎢                                                           ∂               ∂ 
⎢                                                       - 2⋅──(Q₂(x, y, z))⋅──
⎢                                                           ∂x              ∂x
⎢                                                                             
⎢                                                           ∂               ∂ 
⎢                                                       - 2⋅──(Q₃(x, y, z))⋅──
⎢                                                           ∂x              ∂x
⎢                                                                             
⎢  ∂               ∂                    ∂           

In [14]:
E2 = zeros(vec_dim, 1)
for i in range(vec_dim):
    E2[i, 0] = -div_Phi_i[i].ip(div_Q)
    
E2 = simplify(E2)
display(E2)

⎡  ⎛∂                 ∂                 ∂              ⎞ ∂                    
⎢- ⎜──(Q₁(x, y, z)) + ──(Q₂(x, y, z)) + ──(Q₃(x, y, z))⎟⋅──(\phiᵢ(x, y, z)) - 
⎢  ⎝∂x                ∂y                ∂z             ⎠ ∂x                   
⎢                                                                             
⎢           ⎛∂                 ∂                 ∂              ⎞ ∂           
⎢         - ⎜──(Q₁(x, y, z)) + ──(Q₂(x, y, z)) + ──(Q₃(x, y, z))⎟⋅──(\phiᵢ(x, 
⎢           ⎝∂x                ∂y                ∂z             ⎠ ∂y          
⎢                                                                             
⎢  ⎛∂                 ∂                 ∂              ⎞ ∂                    
⎢- ⎜──(Q₁(x, y, z)) + ──(Q₂(x, y, z)) + ──(Q₃(x, y, z))⎟⋅──(\phiᵢ(x, y, z)) + 
⎢  ⎝∂x                ∂y                ∂z             ⎠ ∂z                   
⎢                                                                             
⎢  ⎛∂                 ∂                 ∂           

In [15]:
E31 = zeros(vec_dim, 1)
for i in range(vec_dim):
    E31[i, 0] = -grad_Phi_i[i].ip(Q * grad_Q)
    
E31 = simplify(E31)
display(E31)

⎡  ⎛                              ⎛∂                 ∂              ⎞   ⎛∂    
⎢- ⎜- (Q₁(x, y, z) + Q₄(x, y, z))⋅⎜──(Q₁(x, y, z)) + ──(Q₄(x, y, z))⎟ + ⎜──(Q₁
⎢  ⎝                              ⎝∂z                ∂z             ⎠   ⎝∂x   
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢  ⎛                              ⎛∂                

In [16]:
E32 = zeros(vec_dim, 1)
for i in range(vec_dim):
    E32[i, 0] = Rational(1, 2) * Phi_i[i].ip(grad_Q ** permutedims(grad_Q, (1, 2, 0)))
    
E32 = simplify(E32)
display(E32)

⎡⎛                 2                                                      2   
⎢⎜⎛∂              ⎞    ∂               ∂                 ⎛∂              ⎞    
⎢⎜⎜──(Q₁(x, y, z))⎟  + ──(Q₁(x, y, z))⋅──(Q₄(x, y, z)) - ⎜──(Q₁(x, y, z))⎟  - 
⎢⎝⎝∂x             ⎠    ∂x              ∂x                ⎝∂z             ⎠    
⎢                                                                             
⎢              ⎛⎛∂                 ∂              ⎞ ⎛∂                 ∂      
⎢              ⎜⎜──(Q₁(x, y, z)) + ──(Q₄(x, y, z))⎟⋅⎜──(Q₁(x, y, z)) + ──(Q₄(x
⎢              ⎝⎝∂x                ∂x             ⎠ ⎝∂y                ∂y     
⎢                                                                             
⎢              ⎛⎛∂                 ∂              ⎞ ⎛∂                 ∂      
⎢              ⎜⎜──(Q₁(x, y, z)) + ──(Q₄(x, y, z))⎟⋅⎜──(Q₁(x, y, z)) + ──(Q₄(x
⎢              ⎝⎝∂x                ∂x             ⎠ ⎝∂z                ∂z     
⎢                                                   

## Calculate each of the terms in the Jacobian matrix

In [17]:
dR1 = zeros(vec_dim, vec_dim)
for i in range(vec_dim):
    for j in range(vec_dim):
        dR1[i, j] = Phi_i[i].ip(Phi_j[j])

dR1 = simplify(dR1)
display(dR1)

⎡2⋅\phiᵢ(x, y, z)⋅\phi_j(x, y, z)                 0                           
⎢                                                                             
⎢               0                  2⋅\phiᵢ(x, y, z)⋅\phi_j(x, y, z)           
⎢                                                                             
⎢               0                                 0                  2⋅\phiᵢ(x
⎢                                                                             
⎢ \phiᵢ(x, y, z)⋅\phi_j(x, y, z)                  0                           
⎢                                                                             
⎣               0                                 0                           

      0                   \phiᵢ(x, y, z)⋅\phi_j(x, y, z)                  0   
                                                                              
      0                                 0                                 0   
                                                   

In [18]:
dE1 = zeros(vec_dim, vec_dim)
for i in range(vec_dim):
    for j in range(vec_dim):
        dE1[i, j] = grad_Phi_i[i].ip(grad_Phi_j[j])
display(dE1)

⎡  ∂                  ∂                       ∂                  ∂            
⎢2⋅──(\phiᵢ(x, y, z))⋅──(\phi_j(x, y, z)) + 2⋅──(\phiᵢ(x, y, z))⋅──(\phi_j(x, 
⎢  ∂x                 ∂x                      ∂y                 ∂y           
⎢                                                                             
⎢                                                                             
⎢                                                              0              
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                              0              
⎢                                                                             
⎢                                                                             
⎢   ∂                  ∂                     ∂      

In [19]:
dE2 = zeros(vec_dim, vec_dim)
for i in range(vec_dim):
    for j in range(vec_dim):
        dE2[i, j] = div_Phi_i[i].ip(div_Phi_j[j])
        
dE2 = simplify(dE2)
display(dE2)

⎡ ∂                  ∂                     ∂                  ∂               
⎢ ──(\phiᵢ(x, y, z))⋅──(\phi_j(x, y, z)) + ──(\phiᵢ(x, y, z))⋅──(\phi_j(x, y, 
⎢ ∂x                 ∂x                    ∂z                 ∂z              
⎢                                                                             
⎢                     ∂                  ∂                                    
⎢                     ──(\phiᵢ(x, y, z))⋅──(\phi_j(x, y, z))                  
⎢                     ∂y                 ∂x                                   
⎢                                                                             
⎢  ∂                  ∂                     ∂                  ∂              
⎢- ──(\phiᵢ(x, y, z))⋅──(\phi_j(x, y, z)) + ──(\phiᵢ(x, y, z))⋅──(\phi_j(x, y,
⎢  ∂x                 ∂z                    ∂z                 ∂x             
⎢                                                                             
⎢                     ∂                  ∂          

In [20]:
dE31 = zeros(vec_dim, vec_dim)
for i in range(vec_dim):
    for j in range(vec_dim):
        dE31[i, j] = -grad_Phi_i[i].ip(Phi_j[j]*grad_Q + Q*grad_Phi_j[j])
        
dE31 = simplify(dE31)
display(dE31)

⎡    ⎛            ∂                                 ∂                         
⎢- 2⋅⎜Q₂(x, y, z)⋅──(\phi_j(x, y, z)) + Q₄(x, y, z)⋅──(\phi_j(x, y, z)) + Q₅(x
⎢    ⎝            ∂x                                ∂y                        
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [21]:
dE32 = zeros(vec_dim, vec_dim)
for i in range(vec_dim):
    for j in range(vec_dim):
        dE32[i, j] = -Phi_i[i].ip(grad_Phi_j[j] ** permutedims(grad_Q, (1, 2, 0)))
        
dE32 = simplify(dE31)
display(dE31)

⎡    ⎛            ∂                                 ∂                         
⎢- 2⋅⎜Q₂(x, y, z)⋅──(\phi_j(x, y, z)) + Q₄(x, y, z)⋅──(\phi_j(x, y, z)) + Q₅(x
⎢    ⎝            ∂x                                ∂y                        
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

## Automatically generate C++ code

In [22]:
from sympy.printing.cxx import CXX11CodePrinter

### Begin by making lists of objects relevant to dealii

In [49]:
Q_list = list(Q_vec)
Q0_list = list(Q0_vec)
phi_list = list([phi_i, phi_j])
Lambda_list = list(Lambda_vec)
symbol_list = [alpha, dt, L2, L3]
symbol_list_code = ["alpha", "dt", "L2", "L3"]

display(Q_list)
display(Q0_list)
display(phi_list)
display(Lambda_list)
display(symbol_list)

[Q₁(x, y, z), Q₂(x, y, z), Q₃(x, y, z), Q₄(x, y, z), Q₅(x, y, z)]

[Q_{01}(x, y, z), Q_{02}(x, y, z), Q_{03}(x, y, z), Q_{04}(x, y, z), Q_{05}(x,
 y, z)]

[\phiᵢ(x, y, z), \phi_j(x, y, z)]

[\Lambda₁(x, y, z), \Lambda₂(x, y, z), \Lambda₃(x, y, z), \Lambda₄(x, y, z), \
Lambda₅(x, y, z)]

[\alpha, \delta t, L₂, L₃]

In [24]:
grad_Q_vec = zeros(vec_dim, mat_dim)
grad_phi_vec = zeros(2, mat_dim)
grad_Lambda_vec = zeros(vec_dim, mat_dim)
for i in range(vec_dim):
    for j in range(mat_dim):
        grad_Q_vec[i, j] = Q_list[i].diff(xi[j])
        grad_Lambda_vec[i, j] = Lambda_list[i].diff(xi[j])
        
for i in range(2):
    for j in range(mat_dim):
        grad_phi_vec[i, j] = phi_list[i].diff(xi[j])
        
        
grad_Q_vec_list = grad_Q_vec.tolist()
grad_phi_vec_list = grad_phi_vec.tolist()
grad_Lambda_vec_list = grad_Lambda_vec.tolist()

display(grad_Q_vec_list)
display(grad_phi_vec_list)
display(grad_Lambda_vec_list)

⎡⎡∂                ∂                ∂              ⎤  ⎡∂                ∂     
⎢⎢──(Q₁(x, y, z)), ──(Q₁(x, y, z)), ──(Q₁(x, y, z))⎥, ⎢──(Q₂(x, y, z)), ──(Q₂(
⎣⎣∂x               ∂y               ∂z             ⎦  ⎣∂x               ∂y    

           ∂              ⎤  ⎡∂                ∂                ∂             
x, y, z)), ──(Q₂(x, y, z))⎥, ⎢──(Q₃(x, y, z)), ──(Q₃(x, y, z)), ──(Q₃(x, y, z)
           ∂z             ⎦  ⎣∂x               ∂y               ∂z            

 ⎤  ⎡∂                ∂                ∂              ⎤  ⎡∂                ∂  
)⎥, ⎢──(Q₄(x, y, z)), ──(Q₄(x, y, z)), ──(Q₄(x, y, z))⎥, ⎢──(Q₅(x, y, z)), ──(
 ⎦  ⎣∂x               ∂y               ∂z             ⎦  ⎣∂x               ∂y 

              ∂              ⎤⎤
Q₅(x, y, z)), ──(Q₅(x, y, z))⎥⎥
              ∂z             ⎦⎦

⎡⎡∂                   ∂                   ∂                 ⎤  ⎡∂             
⎢⎢──(\phiᵢ(x, y, z)), ──(\phiᵢ(x, y, z)), ──(\phiᵢ(x, y, z))⎥, ⎢──(\phi_j(x, y
⎣⎣∂x                  ∂y                  ∂z                ⎦  ⎣∂x            

       ∂                    ∂                  ⎤⎤
, z)), ──(\phi_j(x, y, z)), ──(\phi_j(x, y, z))⎥⎥
       ∂y                   ∂z                 ⎦⎦

⎡⎡∂                      ∂                      ∂                    ⎤  ⎡∂    
⎢⎢──(\Lambda₁(x, y, z)), ──(\Lambda₁(x, y, z)), ──(\Lambda₁(x, y, z))⎥, ⎢──(\L
⎣⎣∂x                     ∂y                     ∂z                   ⎦  ⎣∂x   

                  ∂                      ∂                    ⎤  ⎡∂           
ambda₂(x, y, z)), ──(\Lambda₂(x, y, z)), ──(\Lambda₂(x, y, z))⎥, ⎢──(\Lambda₃(
                  ∂y                     ∂z                   ⎦  ⎣∂x          

           ∂                      ∂                    ⎤  ⎡∂                  
x, y, z)), ──(\Lambda₃(x, y, z)), ──(\Lambda₃(x, y, z))⎥, ⎢──(\Lambda₄(x, y, z
           ∂y                     ∂z                   ⎦  ⎣∂x                 

    ∂                      ∂                    ⎤  ⎡∂                      ∂  
)), ──(\Lambda₄(x, y, z)), ──(\Lambda₄(x, y, z))⎥, ⎢──(\Lambda₅(x, y, z)), ──(
    ∂y                     ∂z                   ⎦  ⎣∂x                     ∂y 

                    ∂                    ⎤⎤
\Lam

## Create `MyPrinter` class which extends C++ printer to do necessary substitutions

In [50]:
class MyPrinter(CXX11CodePrinter):
    def _print_Function(self, function):
        if function in Q_list:
            idx = Q_list.index(function)
            return self._print('Q_vec[q][{}]'.format(idx))
        if function in Q0_list:
            idx = Q0_list.index(function)
            return self._print('Q0_vec[q][{}]'.format(idx))
        if function in phi_list:
            idx = phi_list.index(function)
            if idx == 0:
                return self._print('fe_values.shape_value(i, q)')
            elif idx == 1:
                return self._print('fe_values.shape_value(j, q)')
            else:
                raise ValueError("Index out of bounds for phi")
        if function in Lambda_list:
            idx = Lambda_list.index(function)
            return self._print('Lambda_vec[q][{}]'.format(idx))
        
        return super()._print_Function(function)
        
    def _print_Derivative(self, derivative):
        for i, sublist in enumerate(grad_Q_vec_list):
            if derivative in sublist:
                j = sublist.index(derivative)
                return self._print('dQ[q][{}][{}]'.format(i, j))
        for i, sublist in enumerate(grad_phi_vec_list):
            if derivative in sublist:
                j = sublist.index(derivative)
                if i == 0:
                    return self._print('fe_values.shape_grad(q, i)[{}]'.format(j))
                elif i == 1:
                    return self._print('fe_values.shape_grad(q, j)[{}]'.format(j))
                else:
                    raise ValueError("Index out of bounds for dphi")
        for i, sublist in enumerate(grad_Lambda_vec_list):
            if derivative in sublist:
                j = sublist.index(derivative)
                return self._print('dLambda[q][{}][{}]'.format(i, j))
            
        return super()._print_Derivative(derivative)
    
    def _print_Symbol(self, symbol):
        if symbol in symbol_list:
            i = symbol_list.index(symbol)
            return self._print(symbol_list_code[i])
        return super()._print_Symbol(symbol)
        

In [51]:
my_printer = MyPrinter()

## Print terms for isotropic elasticity

In [65]:
assign_string = "    cell_rhs(i) += "
alen = len(assign_string)
for i in range(vec_dim):
    if i == 0:
        print("if (component_i == {})".format(i))
    else:
        print("else if (component_i == {})".format(i))
        
    print(assign_string, end="")
    print("(", end="")
    
    print("(", end="")
    print(my_printer.doprint(R1[i])
                    .replace("*fe_values", 
                             "\n{}* fe_values".format(" "*(alen + 2))
                            )
          , end=""
         )
    print(")")
    
    print(" "*(alen + 1) + "+")
    
    print(" "*(alen + 1), end="")
    print("(", end="")
    print(my_printer.doprint(R2[i])
                    .replace("*fe_values", 
                             "\n{}* fe_values".format(" "*(alen + 2))
                            )
          , end=""
         )
    print(")")
    
    print(" "*(alen + 1) + "+")
    
    print(" "*(alen + 1), end="")
    print("(", end="")
    print(my_printer.doprint(R3[i])
                    .replace("*fe_values", 
                             "\n{}* fe_values".format(" "*(alen + 2))
                            )
          , end=""
         )
    print(")")
    
    print(" "*(alen + 1) + "+")
    
    print(" "*(alen + 1), end="")
    print("(", end="")
    print(my_printer.doprint(E1[i])
                    .replace(" - ", 
                             "\n{}- ".format(" "*(alen + 2))
                            )
                    .replace(" + ",
                             "\n{}+ ".format(" "*(alen + 2))
                            )
          , end=""
         )
    print(")")
    
    print("{})".format(" "*(alen)))
    print("{}* fe_values.JxW(q)".format(" "*(alen)))

if (component_i == 0)
    cell_rhs(i) += (((2*Q_vec[q][0] + Q_vec[q][3])
                     * fe_values.shape_value(i, q))
                    +
                    ((alpha*dt + 1)*(-2*Q0_vec[q][0] - Q0_vec[q][3])
                     * fe_values.shape_value(i, q))
                    +
                    (dt*(2*Lambda_vec[q][0] + Lambda_vec[q][3])
                     * fe_values.shape_value(i, q))
                    +
                    (-2*dQ[q][0][0]*fe_values.shape_grad(q, i)[0]
                     - 2*dQ[q][0][1]*fe_values.shape_grad(q, i)[1]
                     - 2*dQ[q][0][2]*fe_values.shape_grad(q, i)[2]
                     - dQ[q][3][0]*fe_values.shape_grad(q, i)[0]
                     - dQ[q][3][1]*fe_values.shape_grad(q, i)[1]
                     - dQ[q][3][2]*fe_values.shape_grad(q, i)[2])
                   )
                   * fe_values.JxW(q)
else if (component_i == 1)
    cell_rhs(i) += ((2*Q_vec[q][1]
                     * fe_values.shape_value(i, q))
   

In [28]:
for i in range(vec_dim):
    for j in range(vec_dim):
        if (i == 0) and (j == 0):
            print("if (component_i == {} || component_j == {})".format(i, j))
        else:
            print("else if (component_i == {} || component_j == {})".format(i, j))
            
        print("    ", end="")
        print(my_printer.doprint(dR1[i, j]))

if (component_i == 0 || component_j == 0)
    2*fe_values.shape_value(i, q)*fe_values.shape_value(j, q)
else if (component_i == 0 || component_j == 1)
    0
else if (component_i == 0 || component_j == 2)
    0
else if (component_i == 0 || component_j == 3)
    fe_values.shape_value(i, q)*fe_values.shape_value(j, q)
else if (component_i == 0 || component_j == 4)
    0
else if (component_i == 1 || component_j == 0)
    0
else if (component_i == 1 || component_j == 1)
    2*fe_values.shape_value(i, q)*fe_values.shape_value(j, q)
else if (component_i == 1 || component_j == 2)
    0
else if (component_i == 1 || component_j == 3)
    0
else if (component_i == 1 || component_j == 4)
    0
else if (component_i == 2 || component_j == 0)
    0
else if (component_i == 2 || component_j == 1)
    0
else if (component_i == 2 || component_j == 2)
    2*fe_values.shape_value(i, q)*fe_values.shape_value(j, q)
else if (component_i == 2 || component_j == 3)
    0
else if (component_i == 2 || component_j 

In [29]:
first_line = False
for i in range(vec_dim):
    for j in range(vec_dim):
        if dE31[i, j] == 0:
            continue
        if not first_line:
            print("if (component_i == {} || component_j == {})".format(i, j))
            first_line = True
        else:
            print("else if (component_i == {} || component_j == {})".format(i, j))
            
        print("    (", end="")
        print(my_printer.doprint(dE31[i, j]).replace("+", "\n     +").replace("-", "\n     -"))
        print("    )")

if (component_i == 0 || component_j == 0)
    (
     -2*(Q_vec[q][1]*fe_values.shape_grad(q, j)[0] 
     + Q_vec[q][3]*fe_values.shape_grad(q, j)[1] 
     + Q_vec[q][4]*fe_values.shape_grad(q, j)[2])*fe_values.shape_grad(q, i)[1] 
     + ((Q_vec[q][0] 
     + Q_vec[q][3])*fe_values.shape_grad(q, j)[2] 
     + (dQ[q][0][2] 
     + dQ[q][3][2])*fe_values.shape_value(j, q) 
     - Q_vec[q][2]*fe_values.shape_grad(q, j)[0] 
     - Q_vec[q][4]*fe_values.shape_grad(q, j)[1])*fe_values.shape_grad(q, i)[2] 
     + ((Q_vec[q][0] 
     + Q_vec[q][3])*fe_values.shape_grad(q, j)[2] 
     - Q_vec[q][2]*fe_values.shape_grad(q, j)[0] 
     - Q_vec[q][4]*fe_values.shape_grad(q, j)[1] 
     + fe_values.shape_value(j, q)*dQ[q][0][2])*fe_values.shape_grad(q, i)[2] 
     - ((dQ[q][0][0] 
     + dQ[q][3][0])*fe_values.shape_value(j, q) 
     + Q_vec[q][0]*fe_values.shape_grad(q, j)[0] 
     + Q_vec[q][1]*fe_values.shape_grad(q, j)[1] 
     + Q_vec[q][2]*fe_values.shape_grad(q, j)[2])*fe_values.shape_grad(q

In [30]:
print(my_printer.doprint(E1[4]))

-2*dQ[q][4][0]*fe_values.shape_grad(q, i)[0] - 2*dQ[q][4][1]*fe_values.shape_grad(q, i)[1] - 2*dQ[q][4][2]*fe_values.shape_grad(q, i)[2]


In [31]:
CXX11CodePrinter._print

<function sympy.printing.printer.Printer._print(self, expr, **kwargs)>